# BERTopic years' trend

This script aims to generate latent topics in each year from input data sets.

Input file: csv files for the sentiment and emotion analysis.
Output file:

1.   topic infomation in each year including topic representative documents and keywords
2.   corresponding models



**NOTE:**
The script is adapted from https://colab.research.google.com/drive/1BoQ_vakEVtojsd2x_U6-_x52OOuqruj2?usp=sharing#scrollTo=Fo-Oig4Yib5K

In acknowledgment of the contributions made, portions of this code were developed with the guidance and assistance of ChatGPT.


# Enabling the GPU

First, enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [ ]:
%%capture
!pip install bertopic
!pip install sentence-transformers
!pip install umap-learn

import pandas as pd
import matplotlib.pyplot as plt

## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Training

## **Data**



In [ ]:
# Parameters
corpus = "aca" # 🟡Only change this one
min_cluster_size = 75 # 🟡Only change this one
size = str(min_cluster_size)

# input data
from google.colab import drive
drive.mount("/content/drive")
input_path = f"/content/drive/MyDrive/Colab Notebooks/Masters_Thesis/0_corpus/preprocessed_for_sentiment_analysis/{corpus}_sentiment_df.csv"
dataset = pd.read_csv(input_path)

# output paths
output_csv_path = f"/content/drive/MyDrive/Colab Notebooks/Masters_Thesis/5_results/{corpus}_{size}_yearly_trend_topic_info.csv"
output_model_path = f"/content/drive/MyDrive/Colab Notebooks/Masters_Thesis/5_results/"
output_full_csv_path = f"/content/drive/MyDrive/Colab Notebooks/Masters_Thesis/5_results/{corpus}_{size}_text_topic_label_prob.csv"
output_yearly_trend_per_topic_path = f"/content/drive/MyDrive/Colab Notebooks/Masters_Thesis/5_results/{corpus}_{size}_yearly_trend_per_topic.csv"


# Extract abstracts to train on and corresponding titles
abstracts = dataset["text"]
abstracts = abstracts.fillna("")
abstracts = abstracts.astype(str)

# # Load the model, if you have already got a trained model
# from sentence_transformers import SentenceTransformer
# from bertopic import BERTopic

# path = "/content/drive/MyDrive/Colab Notebooks/Masters_Thesis/5_results/model"
# # Define embedding model
# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# # Load model and add embedding model
# topic_model = BERTopic.load(path, embedding_model=embedding_model)

In [ ]:
abstracts[0]

"Changing the World One Meme at a Time: The Effects of Climate Change Memes on Civic Engagement Intentions. The risks that animals and humans face from climate change are frequently featured in climate change memes. Although memes may appear to some as silly jokes, little is known of the impact they may have in the formation and spread of individuals' perceived risk of climate change and intention to participate in climate change campaigns. This study tests the effect of climate change memes on the perceived risk of climate change and the intention of online engagement regarding climate change issues. Results show that exposure to climate change memes increases individual intentions of online civic engagement regarding climate change. Additionally, empathy is found to mediate this effect. However, risk perception of climate change is not altered after exposure to climate change memes."

## **Pre-calculate Embeddings**
After having created our data, namely `abstracts`, we can dive into the very first best practice, **pre-calculating embeddings**.

BERTopic works by converting documents into numerical values, called embeddings. This process can be very costly, especially if we want to iterate over parameters. Instead, we can calculate those embeddings once and feed them to BERTopic to skip calculating embeddings each time.

In [ ]:
from sentence_transformers import SentenceTransformer

# Pre-calculate embeddings
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(abstracts, show_progress_bar=True)

Batches:   0%|          | 0/454 [00:00<?, ?it/s]

## **Preventing Stochastic Behavior**
In BERTopic, we generally use a dimensionality reduction algorithm to reduce the size of the embeddings. This is done to prevent the [curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality) to a certain degree.

As a default, this is done with [UMAP](https://github.com/lmcinnes/umap) which is an incredible algorithm for reducing dimensional space. However, by default, it shows stochastic behavior which creates different results each time you run it. To prevent that, we will need to set a `random_state` of the model before passing it to BERTopic.

As a result, we can now fully reproduce the results each time we run the model.

In [ ]:
from umap import UMAP

umap_model = UMAP(n_neighbors=15, n_components=5, min_dist=0.0, metric="cosine", random_state=42)

## **Controlling Number of Topics**
There is a parameter to control the number of topics, namely `nr_topics`. This parameter, however, merges topics **after** they have been created. It is a parameter that supports creating a fixed number of topics.

However, it is advised to control the number of topics through the cluster model which is by default HDBSCAN. HDBSCAN has a parameter, namely `min_topic_size` that indirectly controls the number of topics that will be created.

A higher `min_topic_size` will generate fewer topics and a lower `min_topic_size` will generate more topics.

Here, we will go with `min_topic_size=40` to get around XXX topics.

In [ ]:
from hdbscan import HDBSCAN

hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, metric="euclidean", cluster_selection_method="eom", prediction_data=True)

## **Improving Default Representation**
The default representation of topics is calculated through [c-TF-IDF](https://maartengr.github.io/BERTopic/algorithm/algorithm.html#5-topic-representation). However, c-TF-IDF is powered by the [CountVectorizer](https://maartengr.github.io/BERTopic/getting_started/vectorizers/vectorizers.html) which converts text into tokens. Using the CountVectorizer, we can do a number of things:

* Remove stopwords
* Ignore infrequent words
* Increase

In other words, we can preprocess the topic representations **after** documents are assigned to topics. This will not influence the clustering process in any way.

Here, we will ignore English stopwords and infrequent words. Moreover, by increasing the n-gram range we will consider topic representations that are made up of one or two words.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text

stop_words = text.ENGLISH_STOP_WORDS.union(["ve", "ha","don","did","ll",
                                            "climate", "change", "just",
                                            "like","think","really","going"])
stop_words = list(stop_words)

vectorizer_model = CountVectorizer(stop_words=stop_words, min_df=5, ngram_range=(1, 2))

In [ ]:
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
print(ENGLISH_STOP_WORDS)

frozenset({'however', 'name', 'show', 'often', 'we', 'further', 'from', 'on', 'him', 'somehow', 'former', 'four', 'another', 'why', 'with', 'among', 'its', 'never', 'they', 'nine', 'wherever', 'at', 'become', 'been', 'something', 'already', 'someone', 'became', 'seeming', 'due', 'yet', 'none', 'amoungst', 'upon', 'namely', 'after', 'down', 'even', 'somewhere', 'hence', 'amongst', 'ten', 'be', 'nowhere', 'where', 'when', 'her', 'full', 'by', 'any', 'onto', 'herein', 'should', 'beforehand', 'what', 'whenever', 'alone', 'system', 'again', 'it', 'whither', 'than', 'below', 'against', 'no', 'whether', 'yourselves', 'within', 'thin', 'this', 'himself', 'to', 'else', 'de', 'side', 'do', 'a', 'are', 'three', 'thereafter', 'see', 'here', 'may', 'otherwise', 'their', 'without', 'hereby', 'his', 'as', 'too', 'many', 'hereupon', 'anything', 'around', 'becomes', 'part', 'per', 'ltd', 'neither', 'you', 'meanwhile', 'put', 'done', 'us', 'elsewhere', 'that', 'or', 'your', 'latter', 'almost', 'fifteen'

## **Additional Representations**
Previously, we have tuned the default representation but there are quite a number of [other topic representations](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html) in BERTopic that we can choose from. From [KeyBERTInspired](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html#keybertinspired) and [PartOfSpeech](https://maartengr.github.io/BERTopic/getting_started/representation/representation.html#partofspeech), to [OpenAI"s ChatGPT](https://maartengr.github.io/BERTopic/getting_started/representation/llm.html#chatgpt) and [open-source](https://maartengr.github.io/BERTopic/getting_started/representation/llm.html#langchain) alternatives, many representations are possible.

In BERTopic, you can model many different topic representations simultanously to test them out and get different perspectives of topic descriptions. This is called [multi-aspect](https://maartengr.github.io/BERTopic/getting_started/multiaspect/multiaspect.html) topic modeling.

Here, we will demonstrate a number of interesting and useful representations in BERTopic:

* KeyBERTInspired
  * A method that derives inspiration from how KeyBERT works
* PartOfSpeech
  * Using SpaCy"s POS tagging to extract words
* MaximalMarginalRelevance
  * Diversify the topic words
* OpenAI
  * Use ChatGPT to label our topics


In [ ]:
from bertopic.representation import KeyBERTInspired, MaximalMarginalRelevance, PartOfSpeech

# KeyBERT
keybert_model = KeyBERTInspired()

# Part-of-Speech
pos_model = PartOfSpeech("en_core_web_sm")

# MMR
mmr_model = MaximalMarginalRelevance(diversity=0.3)

# All representation models
representation_model = {
    "KeyBERT": keybert_model,
    "MMR": mmr_model,
    "POS": pos_model
}

## **Training**
Now that we have a set of best practices, we can use them in our training loop. Here, several different representations, keywords and labels for our topics will be created. If you want to iterate over the topic model it is advised to use the pre-calculated embeddings as that significantly speeds up training.

In [ ]:
from bertopic import BERTopic

"""
# If the results do not make sense, then change this parameter and run it again.
min_cluster_size = 75
size = str(min_cluster_size)
hdbscan_model = HDBSCAN(min_cluster_size=min_cluster_size, metric="euclidean", cluster_selection_method="eom", prediction_data=True)
output_csv_path = f"/content/drive/MyDrive/Colab Notebooks/Masters_Thesis/5_results/{corpus}_{size}_yearly_trend_topic_info.csv"
output_model_path = f"{corpus}_model_year_trend"
"""

topic_model = BERTopic(

  # Pipeline models
  embedding_model=embedding_model,
  umap_model=umap_model,
  hdbscan_model=hdbscan_model,
  vectorizer_model=vectorizer_model,
  representation_model=representation_model,

  # Hyperparameters
  top_n_words=10,
  verbose=True,
  calculate_probabilities=True # show probs of all topics for each text
)

topics, probs = topic_model.fit_transform(abstracts, embeddings)

2024-01-15 16:25:04,053 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-01-15 16:25:35,864 - BERTopic - Dimensionality - Completed ✓
2024-01-15 16:25:35,866 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-01-15 16:25:37,651 - BERTopic - Cluster - Completed ✓
2024-01-15 16:25:37,663 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-01-15 16:25:54,067 - BERTopic - Representation - Completed ✓


In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,KeyBERT,MMR,POS,Representative_Docs
0,-1,4408,-1_adaptation_policy_research_impacts,"[adaptation, policy, research, impacts, study,...","[warming, environmental, mitigation, emissions...","[adaptation, policy, research, impacts, study,...","[adaptation, policy, research, impacts, study,...","[Integrating Technologies, Measures and Polici..."
1,0,1818,0_species_forest_distribution_future,"[species, forest, distribution, future, biodiv...","[species distribution, forest management, tree...","[species, forest, distribution, future, biodiv...","[species, forest, future, biodiversity, conser...",[Assessing potential effects of land use and c...
2,1,1816,1_water_model_future_precipitation,"[water, model, future, precipitation, river, r...","[hydrological, hydrology, river basin, precipi...","[water, model, future, precipitation, river, r...","[water, model, future, precipitation, river, r...",[Land Use and Climate Change Impacts on the Hy...
3,2,963,2_farmers_adaptation_strategies_agricultural,"[farmers, adaptation, strategies, agricultural...","[smallholder farmers, agriculture, agricultura...","[farmers, adaptation, strategies, agricultural...","[farmers, adaptation, strategies, agricultural...",[Adaptation strategies by paddy-growing farmer...
4,3,752,3_crop_yield_production_wheat,"[crop, yield, production, wheat, agricultural,...","[crop yields, crop production, crops, agricult...","[crop, yield, production, wheat, agricultural,...","[crop, yield, production, wheat, agricultural,...",[Diverse effects of crop distribution and clim...
5,4,672,4_adaptation_urban_cities_planning,"[adaptation, urban, cities, planning, local, c...","[urban planning, municipalities, local governm...","[adaptation, urban, cities, planning, local, c...","[adaptation, urban, cities, planning, local, c...",[SPATIAL PLANNING FOR ADAPTATING COMPACT URBAN...
6,5,580,5_health_diseases_disease_public health,"[health, diseases, disease, public health, pub...","[health impacts, health risks, public health, ...","[health, diseases, disease, public health, pub...","[health, diseases, disease, public health, pub...",[Public Health Measures to Address the Impact ...
7,6,482,6_marine_fisheries_fish_species,"[marine, fisheries, fish, species, ocean, fish...","[marine ecosystems, fisheries, fishery, ocean ...","[marine, fisheries, fish, species, ocean, fish...","[marine, fisheries, fish, species, ocean, fish...",[Global assessment of marine and freshwater re...
8,7,477,7_students_education_science_school,"[students, education, science, school, childre...","[environmental education, students, study, tea...","[students, education, science, school, childre...","[students, education, science, school, childre...",[Increasing Environmental Action Through Clima...
9,8,355,8_beliefs_public_political_support,"[beliefs, public, political, support, attitude...","[political orientation, global warming, politi...","[beliefs, public, political, support, attitude...","[beliefs, public, political, support, attitude...",[Their Economy and Our Health: Communicating C...


Save the results

In [ ]:
topic_info = topic_model.get_topic_info()
topic_info_df = pd.DataFrame(topic_info)
topic_info_df.to_csv(output_csv_path, index=False)

To get all representations for a single topic, we simply run the following:

In [ ]:
topic_model.get_topic(0, full=True)

{'Main': [('species', 0.05558441131309337),
  ('forest', 0.029782156110821058),
  ('distribution', 0.01832556243526269),
  ('future', 0.01684363816083875),
  ('biodiversity', 0.016339688345807415),
  ('conservation', 0.01630621616093892),
  ('habitat', 0.015575962375380142),
  ('range', 0.01524021782074512),
  ('forests', 0.01493177833923872),
  ('management', 0.01492696411674417)],
 'KeyBERT': [('species distribution', 0.5216267),
  ('forest management', 0.4668277),
  ('tree species', 0.46404606),
  ('biodiversity', 0.45160633),
  ('species', 0.4493087),
  ('forests', 0.42364168),
  ('habitats', 0.42255327),
  ('ecosystem', 0.42064992),
  ('ecological', 0.40794584),
  ('ecosystems', 0.3981977)],
 'MMR': [('species', 0.05558441131309337),
  ('forest', 0.029782156110821058),
  ('distribution', 0.01832556243526269),
  ('future', 0.01684363816083875),
  ('biodiversity', 0.016339688345807415),
  ('conservation', 0.01630621616093892),
  ('habitat', 0.015575962375380142),
  ('range', 0.01524

**NOTE**: The labels generated by OpenAI"s **ChatGPT** are especially interesting to use throughout your model. Below, we will go into more detail how to set that as a custom label.

# Presenting

## **(Custom) Labels**
The default label of each topic are the top 3 words in each topic combined with an underscore between them.

This, of course, might not be the best label that you can think of for a certain topic. Instead, we can use `.set_topic_labels` to manually label all or certain topics.

We can also use `.set_topic_labels` to use one of the other topic representations that we had before, like `KeyBERTInspired` or even `OpenAI`.

In [ ]:
# label dictionary
topic_to_label = {
    -1: "Noise",
    0: "Biodiversity", 1: "Hydrology", 2: "Agriculture and Farmers",
    3: "Crops", 4: "Urban Planning", 5: "Public Health Risks",
    6: "Ocean Ecosystems", 7: "Education", 8: "Political Influences",
    9: "Indigenous Perspectives", 10: "Vegetation", 11: "Human Rights",
    12: "Tourism", 13: "Media", 14: "Business Management",
    15: "International Climate Governance Developments", 16: "Human Migration",
    17: "Building Energy Use", 18: "Economic Impacts", 19: "Energy",
    20: "Mental Health", 21: "Sociology's Role in Climate Change Research",
    22: "Infrastructure Risk Management"
}

# Label the topics yourself
topic_model.set_topic_labels(topic_to_label)

# Update the dataset and map the labels to topics
dataset["topic"] = topics
dataset["probs"] = probs.tolist()
dataset["label"] = dataset["topic"].map(topic_to_label)

# Check the results
print(dataset.head())

# Save the csv
dataset.to_csv(output_full_csv_path, index=False)

**🔥 Tip - Parameters 🔥**
***
If you would like to return the topic-document probability matrix, then it is advised to use `calculate_probabilities=True`. Do note that this can significantly slow down training. To speed it up, use [cuML's HDBSCAN](https://maartengr.github.io/BERTopic/getting_started/clustering/clustering.html#cuml-hdbscan) instead. You could also approximate the topic-document probability matrix with `.approximate_distribution` which will be discussed later.
***

## Topical Trend Anslysis

### Calculate yearly average probs per topic and define functions

In [ ]:
# creat a hash to save the topic weight distribution of each text and the total num of texts
# if a text is assigned to the Topic -1 (Noise), I still count its disctibution (vector)
yearly_topic_probs_sum = {}
yearly_texts_count = {}

for index, row in dataset.iterrows():
    year = row["year"]
    topic_probs = row["probs"]

    # make sure that every year's topic weight disctribution is saved
    if year not in yearly_topic_probs_sum:
        yearly_topic_probs_sum[year] = [0] * len(topic_probs)
        yearly_texts_count[year] = 0

    # sum up the prob of each topic respectively
    yearly_topic_probs_sum[year] = [sum(x) for x in zip(yearly_topic_probs_sum[year], topic_probs)]
    # save the number of texts of each year
    yearly_texts_count[year] += 1

# calculate the avg prob of each topic in every year
yearly_avg_topic_probs = {year: [prob / yearly_texts_count[year] for prob in probs]
                          for year, probs in yearly_topic_probs_sum.items()}


# Save the yearly average topic probs
label_probs_df = pd.DataFrame(list(yearly_avg_topic_probs.items()), columns=["year", "probs"])
label_probs_df.to_csv(output_yearly_trend_per_topic_path, index=False)

## **Serialization**

When saving a BERTopic model, there are several ways in doing so. You can either save the entire model with `pickle`, `pytorch`, or `safetensors`.

Personally, I would advise going with `safetensors` whenever possible. The reason for this is that the format allows for a very small topic model to be saved and shared.

When saving a model with `safetensors`, it skips over saving the dimensionality reduction and clustering models. The `.transform` function will still work without these models but instead assign topics based on the similarity between document embeddings and the topic embeddings.

As a result, the `.transform` step might give different results but it is generally worth it considering the smaller and significantly faster model.

In [ ]:
# Save the model
embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
topic_model.save(output_model_path, serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

**💡  NOTE - Embedding Model 💡**
***
Using `safetensors`, we are not saving the underlying embedding model but merely a pointer to the model. For example, in the above example we are saving the string `"sentence-transformers/all-MiniLM-L6-v2"` so that we can load in the embedding model alongside the topic model.

This currently only works if you are using a sentence transformer model. If you are using a different model, you can load it in when loading the topic model like this:

```python
from sentence_transformers import SentenceTransformer

# Define embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load model and add embedding model
loaded_model = BERTopic.load("path/to/my/model_dir", embedding_model=embedding_model)
```
***

As mentioned above, loading can be done as follows:

In [ ]:
from sentence_transformers import SentenceTransformer

# Define embedding model
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")

# Load model and add embedding model
loaded_model = BERTopic.load("my_model_dir", embedding_model=embedding_model)